# Student Admission Mini Project

This is my own Notebook while finishing Udacity Data Science Nanodegree Mini Project in Deep Learning Part.

The original notebook provided by Udacity is typically follow-along codes, hence I create this notebook to sharpen my understanding and to tweak more on how the model work on the problem.

In [1]:
!tree

.
├── README.md
├── StudentAdmission.ipynb
└── student_data.csv

0 directories, 3 files


## Dataset

The dataset used here comprises three features:

1. GRE Scores (Test)
2. GPA Scores (Grade)
3. Class rank(1-4)

The goal is **to predict student admission to graduate school at UCLA**

## Load The Data

In [2]:
# importing pandas and numpy
import pandas as pd
import numpy as np

# read the data
data = pd.read_csv('./student_data.csv')

# data review
data.head(10)

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4
5,1,760,3.00,2
6,1,560,2.98,1
7,0,400,3.08,2
8,1,540,3.39,3
9,0,700,3.92,2
